### import libs


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow.keras as k

import csv
import cv2
import os

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201 ,DenseNet121
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras import layers


from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split

from scipy import ndimage

In [7]:
imagetype= os.listdir("../input/fer2013/train")
imagetype

In [8]:
test_imagetype= os.listdir("../input/fer2013/test")
test_imagetype

In [9]:
# imagetype_affect= os.listdir("../input/affectnethq")
# imagetype_affect.remove('contempt')
# imagetype_affect.remove('labels.csv')
# imagetype_affect

In [10]:
emotions_labels_dict = {
   'surprise' : 0,
    'fear': 1,
    'neutral': 2,
    'sad': 3,
    'disgust': 4,
    'happy' : 5,
    'anger': 6,
}

In [11]:
images = []    #images for train list will store all images
labels= []
t_images = []    #images for test list will store all images
t_labels= []
path_fer = "../input/fer2013/train/"
path_affect = "../input/affectnethq/"
t_path = "../input/fer2013/test/"


## **FER13**

### *Load data*

In [12]:
for i in imagetype:
    print(i)
    flower_path=path_fer+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".jpg") ]
    counter=0
    for data in data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (64,64))
        images.append(img)
        labels.append(i)
        
#     print(i)    
#     flower_path1=path_affect+str(i)
#     data_list1=[j for j in os.listdir(flower_path1) if j.endswith(".jpg") ]
#     for data in data_list1:
# #         if counter> 2390: break
#         img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (48,48))
#         images.append(img)
#         labels.append(i)
        
        
print("total train images:",len(images))
print("total train images:",len(labels))


In [13]:
for i in imagetype:
    print(i)
    flower_path=t_path+str(i)
    test_data_list=[j for j in os.listdir(flower_path) if j.endswith(".jpg") ]
    counter=0
    for data in test_data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (64,64))
        t_images.append(img)
        t_labels.append(i)
print("total test images:",len(t_images))
print("total test images:",len(t_labels))

In [14]:
type(images)

In [15]:
encode= LabelEncoder()
labels=encode.fit_transform(labels)
encode2= LabelEncoder()
t_labels = encode2.fit_transform(t_labels)
t_labels[1:10]

In [16]:
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
labels
print("train labels is ready!")

In [17]:
t_labels=t_labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
t_labels=onehotencoder.fit_transform(t_labels)
t_labels=t_labels.toarray()
print("test labels is ready!")
t_labels[:10]

In [18]:
images=np.array(images)
print(images.shape)
print(labels.shape)

In [19]:
t_images=np.array(t_images)
print(t_images.shape)
print(t_labels.shape)

In [20]:
images = images/ 255
t_images = t_images/ 255

In [21]:
images,labels = shuffle(images, labels, random_state=1)
t_images , t_labels =  shuffle(t_images, t_labels, random_state=1)

In [22]:
plt.imshow(images[4])
plt.xlabel(labels[4])

## **CKPLUS**

### *load data*

In [ ]:
for i in ck_imagetype:
    print(i)
    flower_path=ck_path+"/"+str(i)
    data_list=[j for j in os.listdir(flower_path) if j.endswith(".png") ]
    counter=0
    for data in data_list:
#         if counter> 2390: break
        img=cv2.imread(flower_path + '/'+ data)
#         img = cv2.resize(img , (64,64))
        images.append(img)
        labels.append(i)
    
print("total train images:",len(images))
print("total train images:",len(labels))

In [ ]:
print("start encoding", end=" ---> ")
encode= LabelEncoder()
labels=encode.fit_transform(labels)
print ("encoding finished")
print ("start one hot encoding", end=" ---> ")
labels=labels.reshape(-1,1)
onehotencoder= OneHotEncoder()
labels=onehotencoder.fit_transform(labels)
labels=labels.toarray()
print("one hot encoding finished")
images=np.array(images)
print("images and list shapes:")
print("images shape:", end=" " )
print(images.shape)
print("labels shape:", end=" " )
print(labels.shape)


In [ ]:
images,labels = shuffle(images, labels, random_state=10)
plt.imshow(images[4])
plt.xlabel(labels[4])

### *DenseNet201*

In [23]:
DenseNet201 = DenseNet201(input_shape= (48,48,3) ,  include_top=False , weights="imagenet", pooling='max')

In [ ]:
DenseNet201.trainable = False
x = Flatten()(DenseNet201.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
DenseNet201 = Model(inputs=DenseNet201.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
DenseNet201.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = DenseNet201.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### *MobileNet*

In [ ]:
mobile = MobileNet(input_shape= (48,48,3) , dropout = 0.1 ,  include_top=False , weights="imagenet", pooling='max')

In [ ]:
mobile.trainable = False
x = Flatten()(mobile.output)
prediction = Dense(7, activation='softmax')(x)
mobile = Model(inputs=mobile.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
mobile.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = mobile.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### *VGG16*

In [ ]:
vgg16 = VGG16(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')


In [ ]:
vgg16.trainable = True
x = Flatten()(vgg16.output)
prediction = Dense(7, activation='softmax')(x)

# create a model object
vgg16 = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
vgg16.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = vgg16.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# vgg16.save_weights('vgg16_ckplus.h5')
# 

## **models**

*NO.1*

In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model.fit(images , labels , batch_size=128 , epochs=30 , validation_data = (t_images ,t_labels))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

#### some change

In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


  #CNN layer 
# 32 ,64 ,128
  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 

  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(16),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.6), 
    
  layers.Dense(8),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.6), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model.fit(images , labels , batch_size=32 , epochs=40 , validation_data = (t_images ,t_labels))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


  #CNN layer 
# 32 ,64 ,128
  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 
    
  layers.Conv2D(64,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 

  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(32),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.6), 
    
  layers.Dense(16),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.6), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
history = model.fit(images , labels , batch_size=32 , epochs=40 , validation_data = (t_images ,t_labels))

*NO.2*

In [ ]:
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
model2 = tf.keras.Sequential()

#Block 1:
model2.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,3)))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
# model2.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
# model2.add(Activation('relu'))
# model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.65))

#Block 2:
model2.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
# model2.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
# model2.add(Activation('relu'))
# model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.65))

# #block 3:
model2.add(Conv2D(512,(3,3),padding='same',kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.65))


model2.add(Flatten())
model2.add(Dense(1024,kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model.add(Dropout(0.65))


model2.add(Dense(1024,kernel_initializer='he_normal'))
model2.add(Activation('relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.65))


model2.add(Dense(7,kernel_initializer='he_normal'))
model2.add(Activation('softmax'))

model2.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model2.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model2.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### VGG16



In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  VGG16(weights='imagenet',include_top=False, classes=6, input_shape=(64,64,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

  layers.Conv2D(64,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.65), 
    
#    layers.Conv2D(128,(3,3),padding = 'same'), 
#   layers.BatchNormalization(),
#   layers.LeakyReLU(alpha=0.1),
#   layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#   layers.Dropout(0.65),
    
#  layers.Conv2D(512,(3,3),padding = 'same'), 
#  layers.BatchNormalization(),
#  layers.LeakyReLU(alpha=0.1),
#  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#  layers.Dropout(0.65), 
    
 layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(1000),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(6, activation='softmax'),
                          
])
model.summary()

In [ ]:
# model.trainable = False
# dropout1 = Dropout(0.5)
# x = Flatten()(model.output)
# x= x = dropout1(model.output)
# prediction = Dense(6, activation='softmax')(x)
# model = Model(inputs=model.input, outputs=prediction)

In [ ]:
# opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
opt = keras.optimizers.Adam(learning_rate=1e-7)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)


In [ ]:
history = model.fit(images , labels, batch_size=128 , epochs=30 , validation_data = (t_images , t_labels))

In [ ]:
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### NASNetLarge

In [ ]:
nas = NASNetLarge(
    input_shape=(64,64,3),
    include_top=True,
#     weights="imagenet",
    pooling='max',
#     classes=6
)

In [ ]:
nas.trainable = False
x = Flatten()(nas.output)
prediction = Dense(7, activation='softmax')(x)

# create a model object
nas = Model(inputs=nas.input, outputs=prediction)

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
nas.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
history = nas.fit(images , labels, batch_size=32 , epochs=30 , validation_data = (t_images , t_labels))

#### efficientNet B0

In [ ]:
ef_b0 = tf.keras.applications.EfficientNetB0(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

In [ ]:
ef_b0.trainable = False
x = Flatten()(ef_b0.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b0 = Model(inputs=ef_b0.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b0.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = ef_b0.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### edit version


In [ ]:
model = keras.Sequential([
# Data Augmentation 
  
 tf.keras.applications.EfficientNetB0(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)),
# Data Augmentation 
  

#   DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),

#CNN layer 

#   layers.Conv2D(256,(3,3),padding = 'same'), 
#   layers.BatchNormalization(),
#   layers.LeakyReLU(alpha=0.1),
#   layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
#   layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(1024),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.7), 

layers.Dense(1024),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.7),
# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
r = model.fit(images , labels , batch_size=32 , epochs=100 , validation_data = (t_images , t_labels))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model = keras.Sequential([
# Data Augmentation 
  
 tf.keras.applications.EfficientNetB0(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)),
# Data Augmentation 
  

#   DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),

#CNN layer 

  layers.Conv2D(256,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.7), 


  layers.Flatten(),
#Fully connected 1st layer 

#   layers.Dense(16),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.7), 

  layers.Dense(16),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.7),
# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)
r = model.fit(images , labels , batch_size=32 , epochs=100 , validation_data = (t_images , t_labels))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
ef_b1 = tf.keras.applications.EfficientNetB1(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b1.trainable = False
x = Flatten()(ef_b1.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b1 = Model(inputs=ef_b1.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b1.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = ef_b1.fit(images[736:] , labels[736:] , batch_size=32 , epochs=50 , validation_data = (images[:245] , labels[:245]))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

#### EfficentNet B2

In [ ]:
ef_b2 = tf.keras.applications.EfficientNetB2(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b2.trainable = False
x = Flatten()(ef_b2.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b2 = Model(inputs=ef_b2.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b2.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = ef_b2.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
# plt.savefig('LossVal_loss_DenseNet201')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
ef_b3 = tf.keras.applications.EfficientNetB3(input_shape= (48,48,3) , include_top=False , weights="imagenet", pooling='max')

################## B1

ef_b3.trainable = False
x = Flatten()(ef_b3.output)
prediction = Dense(6, activation='softmax')(x)

# create a model object
ef_b3 = Model(inputs=ef_b3.input, outputs=prediction)

# learning rate 
opt = keras.optimizers.Adam(learning_rate=0.00010000000474974513)
ef_b3.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [ ]:
r = ef_b3.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))

In [ ]:
!pip install utils


In [ ]:
!pip install livelossplot

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot import PlotLossesKeras
# from livelossplot import PlotLossesTensorFlowKeras
import tensorflow as tf
print("Tensorflow version:", tf.__version__)


In [ ]:
model = keras.Sequential()

# Conv Block 1
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 2
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

# Fully connected Block 1
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected Block 2
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

opt = Adam(lr=0.0005)
# opt = Adam(lr= 0.00010000000474974513)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#### Data Augmentation


In [ ]:
# datagen =  ImageDataGenerator(
#   #preprocessing_function=preprocess_input,
# #   rotation_range=30,
# #   width_shift_range=0.2,
# #   height_shift_range=0.2,
# #   shear_range=0.2,
# #   zoom_range=0.2,
#   horizontal_flip=True
# )

# aug = ImageDataGenerator(rotation_range=45,
#                          width_shift_range=0.2,
#                          color_mode='grayscale',
#                          height_shift_range=0.2) 
# #                          horizontal_flip = False,
# #                          fill_mode='nearrest')

# aug.fit(images)
img_size = 48
batch_size = 64

# Data generator to augment data for training
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("../input/emotion-detection-fer/train", 
                                                    target_size=(img_size,img_size), 
                                                    color_mode='grayscale',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True)

# Data generator to augment data for validation
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_train.flow_from_directory("../input/emotion-detection-fer/test", 
                                                    target_size=(img_size,img_size), 
                                                    color_mode='grayscale',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=False)


In [ ]:
# r = model.fit(images , labels , batch_size=32 , epochs=50 , validation_data = (t_images , t_labels))
epochs = 15
steps_per_epoch= train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("model_weights.h5",monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss' , factor=0.1, patience=2, min_lr=0.00001,model='auto')

callbacks = [PlotLossesKeras(), checkpoint, reduce_lr]


In [ ]:
history = model.fit(
        x= train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks
)

In [ ]:
epochs = 32
steps_per_epoch= train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

checkpoint = ModelCheckpoint("model_weights.h5",monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss' , factor=0.1, patience=2, min_lr=0.00001,model='auto')

callbacks = [PlotLossesKeras(), checkpoint, reduce_lr]

history = model.fit(
        x= train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks
)

In [ ]:
model = keras.Sequential([
# Data Augmentation 
  

  DenseNet121(weights='imagenet',include_top=False, classes=7, input_shape=(48,48,3)), #Extract features 

  layers.BatchNormalization(),


#CNN layer 

  layers.Conv2D(128,(3,3),padding = 'same'), 
  layers.BatchNormalization(),
  layers.LeakyReLU(alpha=0.1),
  layers.MaxPooling2D(pool_size = (2,2),padding = 'same'), 
  layers.Dropout(0.5), 


  layers.Flatten(),
#Fully connected 1st layer 

  layers.Dense(256),
  layers.BatchNormalization(), 
  layers.Activation('relu'), 
  layers.Dropout(0.65), 

# Fully connected layer 2nd layer

#   layers.Dense(128),
#   layers.BatchNormalization(), 
#   layers.Activation('relu'), 
#   layers.Dropout(0.5), 


  layers.Dense(7, activation='softmax'),
                          
])
model.summary()

## DeepFace

In [ ]:
!pip install deepface


In [ ]:
from deepface import DeepFace


In [ ]:
face_analysis = DeepFace.analyze(img_path = "../input/emotion-detection-fer/train/neutral/im1026.png")


In [ ]:
face_analysis